In [1]:
cd ../pathofexile/

/Users/steve/Dropbox/Programming/Path of Exile project/pathofexile


In [2]:
import pathofexile.api as poe
import pathofexile.ladder as ladder
import sqlite3

In [3]:
cd ../src

/Users/steve/Dropbox/Programming/Path of Exile project/src


In [4]:
import poe_utils

reload(poe_utils)

<module 'poe_utils' from 'poe_utils.pyc'>

# Connect to an sqlite table

In [5]:
conn = sqlite3.connect('../data/poe_races.db')
c = conn.cursor()

In [6]:
leagues = poe.get_leagues(league_type='season', season='Season10', compact_info=0, league_limit=1, league_offset=0)
print leagues[0].keys()

[u'startAt', u'description', u'registerAt', u'url', u'endAt', u'id', u'rules', u'event']


In [8]:
print leagues[0]

{u'startAt': u'2015-01-02T16:00:00Z', u'description': u'This is the Season Ten Signature Event.\n\nPlease check the Event Forums for more details and points awarded.', u'registerAt': u'2015-01-02T15:30:00Z', u'url': u'http://pathofexile.com/forum/view-thread/1156124', u'endAt': u'2015-01-02T17:30:00Z', u'id': u'90m Fixed Seed Brutal (S10S049)', u'rules': [{u'description': u'A character killed in Hardcore is moved to its parent league.', u'id': 4, u'name': u'Hardcore'}, {u'description': u'You may not party in this league.', u'id': 24, u'name': u'Solo'}], u'event': True}


In [23]:
poe_utils.all_leagues_to_table(c, conn, 'leagues')
#conn.commit() # To actually push changes to the table

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5222


In [14]:
c.execute('SELECT * FROM leagues')
c.fetchall()

[(u'Standard',
  u'The default game mode.',
  u'2013-01-23T21:00:00Z',
  u'None',
  u'None',
  u'http://pathofexile.com/forum/view-thread/71278',
  u'[]',
  0),
 (u'Hardcore',
  u'A character killed in the Hardcore league is moved to the Standard league.',
  u'2013-01-23T21:00:00Z',
  u'None',
  u'None',
  u'http://pathofexile.com/forum/view-thread/71276',
  u"[{u'description': u'A character killed in Hardcore is moved to its parent league.', u'id': 4, u'name': u'Hardcore'}]",
  0),
 (u'Default097',
  u'The default game mode.',
  u'None',
  u'None',
  u'None',
  u'None',
  u'[]',
  0),
 (u'Hardcore097',
  u'A character killed in the Hardcore league is moved to the Default league.',
  u'None',
  u'None',
  u'None',
  u'None',
  u"[{u'description': u'A character killed in Hardcore is moved to its parent league.', u'id': 4, u'name': u'Hardcore'}]",
  0),
 (u'Jan30 3h Rush HC',
  u'A 3 hour hardcore ladder rush with prizes. See the forum for details.',
  u'2012-01-29T21:00:00Z',
  u'2012-0

In [21]:
league_ids = poe_utils.get_leagues_from_table(c,'leagues')
poe_utils.all_ladders_to_table(c, conn, league_ids, 'ladders', start=5222)
conn.commit()

In [9]:
league_ids = poe_utils.get_leagues_from_table(c,'leagues')
ladder1 = ladder.retrieve(league_ids[0])
ladder2 = ladder.retrieve(unicode(league_ids[3055]))

In [10]:
ladder1

[{u'account': {u'challenges': {u'total': 26},
   u'name': u'Helman',
   u'twitch': {u'name': u'helmannn'}},
  u'character': {u'class': u'Duelist',
   u'experience': 1501660,
   u'level': 22,
   u'name': u'Twitch_Tv___Helmannn'},
  u'dead': False,
  u'online': False,
  u'rank': 1},
 {u'account': {u'challenges': {u'total': 36},
   u'name': u'Morsexier',
   u'twitch': {u'name': u'morsrageng'}},
  u'character': {u'class': u'Marauder',
   u'experience': 1463734,
   u'level': 22,
   u'name': u'ImFineEitherwayDickhead'},
  u'dead': False,
  u'online': False,
  u'rank': 2},
 {u'account': {u'challenges': {u'total': 22}, u'name': u'Throzz'},
  u'character': {u'class': u'Witch',
   u'experience': 1462360,
   u'level': 22,
   u'name': u'Throzz_'},
  u'dead': False,
  u'online': False,
  u'rank': 3},
 {u'account': {u'challenges': {u'total': 0},
   u'name': u'cwu',
   u'twitch': {u'name': u'comewithus'}},
  u'character': {u'class': u'Templar',
   u'experience': 1304147,
   u'level': 22,
   u'name': 

In [22]:
conn.close()

In [25]:
# This checks the number of duplicates
c.execute('''
          SELECT * FROM ladders
          WHERE rowid NOT IN(
             SELECT  MIN(rowid)
             FROM    ladders
             GROUP BY id, account, twitch, challenges, character, rank, class, experience, dead
             );
          ''')
len(c.fetchall())

324884

I had some some duplicate records in the ladders table (324884 to be exact). I removed them using the code below (don't forget to commit!).

In [26]:
c.execute('''
          DELETE FROM ladders
          WHERE rowid NOT IN(
             SELECT  MIN(rowid)
             FROM    ladders
             GROUP BY id, account, twitch, challenges, character, rank, class, experience, dead
             );
          ''')

In [28]:
conn.commit()